In [1]:
val data = sc.textFile("Transactions.csv")
//val data = sc.textFile("Transactions.txt")
case class Ticket(TID: Int, Product: String)
case class Frequency(Source: String, Target: String, Weight: Double)
val kv = data.map(_.split(",")).map(v => Ticket(v(0).toInt, v(1))).cache()

// Convert RDD to a DataFrame
val df = sqlContext.createDataFrame(kv)
df.registerTempTable("transactions")

In [2]:
val prodFreq = sqlContext.sql("""
            SELECT Product, CAST(COUNT(DISTINCT TID) AS Double) as count
            FROM transactions 
            GROUP BY Product"""
        ).toDF()

val totFreq = df.groupBy("TID").count().count()

val productSupport = prodFreq.select(prodFreq("Product"), (prodFreq("count") / totFreq).cast("double").as("Support")).cache()


In [3]:
val prodCart = sqlContext.sql("""
            SELECT 
                tab1.Product AS Source, 
                tab2.Product AS Target, 
                COUNT(DISTINCT tab1.TID) AS Weight 
            FROM transactions tab1 
            JOIN transactions tab2 
            WHERE tab1.TID = tab2.TID AND tab1.Product <> tab2.Product
            GROUP BY tab1.Product, tab2.Product"""
        ).toDF().cache()



In [4]:
val ruleSupport = prodCart.select(
            prodCart("Source"), 
            prodCart("Target"), 
            (prodCart("Weight").cast("double") / totFreq.toDouble).as("Support")
            ).cache()

val ruleConfidence = prodCart.join(prodFreq, prodCart("Source") === prodFreq("Product")).select(
            prodCart("Source"), 
            prodCart("Target"), 
            (prodCart("Weight").cast("double") / prodFreq("count")).as("Confidence")
            ).cache()

In [7]:
val ruleConfSup = ruleSupport.join(ruleConfidence, ruleSupport("Source") === ruleConfidence("Source") and 
    ruleSupport("Target") === ruleConfidence("Target")).select(
        ruleSupport("Source"),
        ruleSupport("Target"),
        ruleSupport("Support"),
        ruleConfidence("Confidence")
    )

In [37]:
//lift(X-->Y) = ( (sup(X U Y)/ N) / (sup(X)/ N*sup(Y)/ N ), 

Name: Syntax Error.
Message: 
StackTrace: 

In [8]:
ruleSupport.registerTempTable("rulesupport")
productSupport.registerTempTable("productsupport")
ruleConfSup.registerTempTable("ruleconfsup")

In [15]:
val queryString = "SELECT rs.Source, rs.Target, rs.Support, rs.Confidence, (rs.Support / "+ totFreq.toString+ " / (ss.Support / (ts.Support * "+ totFreq.toString+ ") / "+ totFreq.toString+ """
        )) AS Lift FROM ruleconfsup AS rs
        JOIN productsupport AS ss ON rs.Source = ss.Product
        JOIN productsupport AS ts ON rs.Target = ts.Product"""


In [16]:
val fullMBA = sqlContext.sql(queryString)

In [17]:
fullMBA.repartition(1).save("mba", "json")